# Part 6 - Federated Learning on MNIST using a CNN

## Upgrade to Federated Learning in 10 Lines of PyTorch + PySyft


### Context 

Federated Learning is a very exciting and upsurging Machine Learning technique that aims at building systems that learn on decentralized data. The idea is that the data remains in the hands of its producer (which is also known as the _worker_), which helps improving privacy and ownership, and the model is shared between workers. One immediate application is for example to predict the next word on your mobile phone when you write text: you don't want the data used for training — i.e. your text messages — to be sent to a central server.

The rise of Federated Learning is therefore tightly connected to the spread of data privacy awareness, and the GDPR in EU which enforces data protection since May 2018 has acted as a catalyst. To anticipate on regulation, large actors like Apple or Google have started investing massively in this technology, especially to protect the mobile users' privacy, but they have not made their tools available. At OpenMined, we believe that anyone willing to conduct a Machine Learning project should be able to implement privacy preserving tools with very little effort. We have built tools for encrypting data in a single line [as mentioned in our blog post](https://blog.openmined.org/training-cnns-using-spdz/) and we now release our Federated Learning framework which leverage the new PyTorch 1.0 version to provide an intuitive interface to building secure and scalable models.

In this tutorial, we'll use directly [the canonical example of training a CNN on MNIST using PyTorch](https://github.com/pytorch/examples/blob/master/mnist/main.py) and show how simple it is to implement Federated Learning with it using our [PySyft library](https://github.com/OpenMined/PySyft/). We will go through each part of the example and underline the code which is changed.

You can also find this material in [our blogpost](https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines).

Authors:
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

**Ok, let's get started!**

### Imports and model specifications

First we make the official imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

And than those specific to PySyft. In particular we define remote workers `alice` and `bob`.

In [2]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


We define the setting of the learning task

In [3]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

print('Use device: ', device, ' kwargs ', kwargs)

# --> ref to: https://github.com/OpenMined/PySyft/issues/1893?fbclid=IwAR1p5mmir4kiD5sBn5DJQXHpVATx210fqUhsu-rrZU6HRW_SXbqC0RuzPOA
# this is used for pysyft
'''
Error be like:
RuntimeError: Expected object of backend CPU but got backend CUDA for argument #2 'source'
'''
torch.set_default_tensor_type(torch.cuda.FloatTensor)
# --> ref to: https://discuss.pytorch.org/t/not-using-multiprocessing-but-getting-cuda-error-re-forked-subprocess/54610/8
# force the start method for test session error prevention
'''
Error be like: 
RuntimeError: Cannot re-initialize CUDA in forked subprocess.
'''
torch.multiprocessing.set_start_method('spawn', force=True)

Use device:  cuda  kwargs  {'num_workers': 1, 'pin_memory': True}


### Data loading and sending to workers
We first load the data and transform the training Dataset into a Federated Dataset split across the workers using the `.federate` method. This federated dataset is now given to a Federated DataLoader. The test dataset remains unchanged.

In [4]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


W1125 21:23:07.501220 140288689858304 dataloader.py:197] The following options are not supported: num_workers: 1, pin_memory: True


### CNN specification
Here we use exactly the same CNN as in the official example.

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Define the train and test functions
For the train function, because the data batches are distributed across `alice` and `bob`, you need to send the model to the right location for each batch. Then, you perform all the operations remotely with the same syntax like you're doing local PyTorch. When you're done, you get back the model updated and the loss to look for improvement.

In [6]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

The test function does not change!

In [7]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Launch the training !

In [ ]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

print('Train with ', device)
for epoch in range(1, args.epochs + 1):
    print('epoch ', epoch)
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

print('Finished all')
if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")



Train with  cuda
epoch  1
Train Epoch: 1 [0/60032 (0%)]	Loss: 2.327633
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.143806
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.817529
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.236438
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.664112
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.620395
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.434443
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.526048
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.349476
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.418135
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.583314
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.308133
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.215876
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.267580
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.297201
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.147105
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.259745
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.141259
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.287070
Train Epoch: 1 [36480/60032 (61%)]	L

Train Epoch: 5 [59520/60032 (99%)]	Loss: 0.054318

Test set: Average loss: 0.0484, Accuracy: 9851/10000 (99%)

epoch  6
Train Epoch: 6 [0/60032 (0%)]	Loss: 0.065971
Train Epoch: 6 [1920/60032 (3%)]	Loss: 0.056549
Train Epoch: 6 [3840/60032 (6%)]	Loss: 0.020597
Train Epoch: 6 [5760/60032 (10%)]	Loss: 0.109694
Train Epoch: 6 [7680/60032 (13%)]	Loss: 0.032592
Train Epoch: 6 [9600/60032 (16%)]	Loss: 0.031951
Train Epoch: 6 [11520/60032 (19%)]	Loss: 0.034297
Train Epoch: 6 [13440/60032 (22%)]	Loss: 0.019117
Train Epoch: 6 [15360/60032 (26%)]	Loss: 0.085293
Train Epoch: 6 [17280/60032 (29%)]	Loss: 0.073307
Train Epoch: 6 [19200/60032 (32%)]	Loss: 0.070616
Train Epoch: 6 [21120/60032 (35%)]	Loss: 0.052738
Train Epoch: 6 [23040/60032 (38%)]	Loss: 0.015525
Train Epoch: 6 [24960/60032 (42%)]	Loss: 0.036877
Train Epoch: 6 [26880/60032 (45%)]	Loss: 0.054575
Train Epoch: 6 [28800/60032 (48%)]	Loss: 0.030769
Train Epoch: 6 [30720/60032 (51%)]	Loss: 0.044610
Train Epoch: 6 [32640/60032 (54%)]	Loss: 0

Et voilà! Here you are, you have trained a model on remote data using Federated Learning!

## One Last Thing
I know there's a question you're dying to ask: **how long does it takes to do Federated Learning compared to normal PyTorch?**

The computation time is actually **less than twice the time** used for normal PyTorch execution! More precisely, it takes 1.9 times longer, which is very little compared to the features we were able to add.

## Conclusion

As you observe, we modified 10 lines of code to upgrade the official Pytorch example on MNIST to a real Federated Learning setting!

Of course, there are dozen of improvements we could think of. We would like the computation to operate in parallel on the workers and to perform federated averaging, to update the central model every `n` batches only, to reduce the number of messages we use to communicate between workers, etc. These are features we're working on to make Federated Learning ready for a production environment and we'll write about them as soon as they are released!

You should now be able to do Federated Learning by yourself! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways! 

### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)